# Salary investigation - Feature engineering and Machine learning

**Still working. For data analysis and visualisation, please check the "detailed_analysis_visual_on_income" notebook**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
# sklearn.set_config(print_changed_only=True)
%matplotlib inline

In [5]:
df = pd.read_csv("data/adult.csv", index_col=0)
df['income'] = df['income'].str.replace(" ", "")
df.head()

df_process = df.copy()
# Think for this data, because the target column (income) is a binary value, using x-y plot is difficult to 
# see the relationship between x and y. I think it is better to use histogram, with y as colors

In [2]:
# Label encoding
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def label_encode(df, X_col, label_encode_col):
# y_encode_col = 'income'

    LEncoder = LabelEncoder()
    OHEncoder = OneHotEncoder()

    X_label_enc = df[X_col]

    for col in label_encode_col:
        # The column names have a space in front. Clear it
    #     cleaned_col = col.replace(' ', '')
    #     print(cleaned_col)

        ## TODO Fixed the empty space in front of the column name
        temp_dummy = pd.get_dummies(X_label_enc[col],prefix=col,prefix_sep='-')
        cleaned_val = [c.replace(' ','') for c in temp_dummy.columns]
        cleaned_cols = {k:v for (k,v) in zip(temp_dummy.columns, cleaned_val)}
    #     print(cleaned_cols)
        temp_dummy.rename(columns=cleaned_cols, inplace=True)
    #     print('temp_dummy.columns:', temp_dummy.head(2))
        X_label_enc.drop(columns=col, inplace=True)
        X_label_enc = X_label_enc.merge(temp_dummy, left_index=True, right_index=True)
    #     print(X_label_enc.columns)

    #     print(X)
    # Use for loop to fit and trasnform each column that needs to be encoded
    #     X_label_enc[col] = LEncoder.fit_transform(X_label_enc[col])
    #     X_label_enc[col] = OHEncoder.fit_transform(X_label_enc[col].reshape(-1,1))


    # print(pd.get_dummies(X_label_enc['workclass']))

    # LEncoder.fit_transform(df[columns[0]])
    y_encode = LEncoder.fit_transform(y)

    print(X_label_enc.columns)
    
    return X_label_enc, y_encode

In [ ]:
label_encode_col = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender']

X_label_enc, y_encode = label_encode(df, X_col, label_encode_col)

print(y_encode)

X_label_enc.head()

In [ ]:
# Native country to be engineered further.
# Maybe use some factors to categorise countries such as: 
# 1. GPP (Gross average per person)
# https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)_per_capita
# 2. Cultural similarities (maybe simply English-speaking and non-English speaking?)
# https://www.economist.com/graphic-detail/2019/12/04/where-are-the-worlds-best-english-speakers
# https://ceoworld.biz/2019/11/05/revealed-the-worlds-best-non-native-english-speaking-countries-2019/
# 3. Geographical location (continent)


# Easy version
US_check = ['united-states', 'outlying-us(guam-usvi-etc)']

# Need to do strip because there is weird space in entries
temp_native_country = df_process['native-country'].apply(lambda x: 1 if x.lower().strip() in US_check else 0)

native_country_dummy = pd.get_dummies(temp_native_country, prefix='native-country: US or not', prefix_sep='-')


df_process = df_process.merge(native_country_dummy, left_index=True, right_index=True)

# print(df_process['native-country'][0].lower().strip())

# print(df_process['native-country'].value_counts())
# print(temp_n_country)
# print(native_country_dummy)
df_process.head()

In [ ]:
# More complicated method
# Get the GDP and English fluency information
dict_country_GDP = {}
dict_country_Eng_fluency = {}

temp_country_GDP # bin this using qcut
temp_Eng_fluency # bin this using qcut

## Functions that perform vertical quartile quantification and then horizontal binning
Since several columns require the same procedure (age, capital-gain and capital-loss), will write functions here
for calling <br>
For capital-gain and capital-loss, because there are too many different horizontal values (different amount of capital-gain/capital-loss, thus, would just use a horizontal qcut and omit the vertical one. <br><br>
Because most people have 0 capital-gain, most quartiles are concentrated close to 0 too.
By setting duplicate drop, don't get to see a lot of different bins even though quartile cut is big

## Capital-gain and capital-loss analysis
There are too many different values for capital-gain and capital-loss (and quite a few outliers).
So similar to age feature engineering, I will use a vertical quartile quantification method to limit the horizontal ranges.
The amount of money is again a continuous value, so might have to consider to similar things as age featuring
where I do the horizontal binning based on levels

In [6]:
def bin_num_col(df, col, q_num, drop_zero=False):
    # binning numerical columns with qcut. col should just be one column string. q_num is the number of bins to be made
    # if drop_zero=True, set zero as one of the bin then drop it, then perform qcut with the rest
    # print(df_process['capital-gain'])
    # df_process['capital-gain'].value_counts() # Most people have 0 capital-gain (perhaps no investment?)
    binned = []
#     temp_df = df.copy()
    
    if drop_zero == True:
        binned.extend([-0.01, 0.01])
        temp_df = df[(df[col] < -0.01) | (df[col] > 0.01)][col]
        print(temp_df)
    else:
        temp_df = df[col]
    
    if col in df.columns:
        ser, temp_binned = pd.qcut(temp_df, q=q_num, duplicates="drop", retbins=True)
        # With retbins=True, pd.qcut returns a tuple whose second element is the bins
        # The first return element is a series while the second one is the break point
        binned.extend(temp_binned)
        print(binned)

        # bin_cap_gain -= 0.01
        # Need to do this because the cut is excluding the left hand side boundary value, which means 0 would not
        # be include in (0 x]

        binned_col = pd.cut(df[col], bins=binned, include_lowest=True)

        binned_dummy = pd.get_dummies(binned_col,prefix= col + '_binned',prefix_sep='-')
#         df.drop(columns=[col], inplace=True)  
        df = df.merge(binned_dummy, left_index=True, right_index=True)
    
    return df

In [7]:
df_process = bin_num_col(df_process, 'capital-gain', 5, drop_zero=True)
df_process = bin_num_col(df_process, 'capital-loss', 5, drop_zero=True)
df_process = bin_num_col(df_process, 'education-num', 5)

df_process.head()

0         2174
8        14084
9         5178
59        5013
60        2407
84       14344
101      15024
105       7688
106      34095
113       4064
132       4386
197      14084
198       7298
206       7298
208      15024
227       1409
230       3674
263       4064
271       1055
272       2407
276       7298
279       7298
281       5178
285      15024
297       3464
300       7688
303       4386
311       7298
329       2050
341       7298
         ...  
32098     7298
32123    13550
32139    15024
32155     1055
32156     7298
32164    10520
32169     4101
32179     5178
32198    20051
32203    15024
32228     7688
32238    99999
32249     7688
32308     3103
32341    15831
32354     6849
32365    15024
32367     2653
32370    27828
32388     1471
32399     7298
32434     7443
32462     7298
32466    15024
32473     1506
32515     3471
32518    99999
32538    15020
32548     1086
32560    15024
Name: capital-gain, Length: 2712, dtype: int64
[-0.01, 0.01, 114.0, 3103.0, 5013.0, 7

,age,workclass,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,...,"capital-loss_binned-(0.01, 155.0]","capital-loss_binned-(155.0, 1617.0]","capital-loss_binned-(1617.0, 1876.0]","capital-loss_binned-(1876.0, 1902.0]","capital-loss_binned-(1902.0, 2001.0]","capital-loss_binned-(2001.0, 4356.0]","education-num_binned-(0.999, 9.0]","education-num_binned-(9.0, 10.0]","education-num_binned-(10.0, 13.0]","education-num_binned-(13.0, 16.0]"
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,...,0,0,0,0,0,0,0,0,1,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,...,0,0,0,0,0,0,0,0,1,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,...,0,0,0,0,0,0,1,0,0,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,...,0,0,0,0,0,0,1,0,0,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,...,0,0,0,0,0,0,0,0,1,0
